In [1]:
import copy
import glob
import importlib
import niftiutils.helper_fxns as hf
import niftiutils.transforms as tr
import niftiutils.masks as masks
import numpy as np
import os
import shutil
from os.path import *
%matplotlib inline
from scipy.ndimage.morphology import binary_closing, binary_opening, binary_dilation
from skimage.morphology import ball, label

C:\Users\Clinton\AppData\Local\conda\conda\envs\old-keras\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [89]:
importlib.reload(hf)

<module 'niftiutils.helper_fxns' from 'c:\\users\\clinton\\documents\\niftiutils\\niftiutils\\helper_fxns.py'>

In [2]:
img_dir = "D:\\Lipiodol\\Images all"
seg_dir = "D:\\Lipiodol\\Images extracted and segmentations"
target_dir = "D:\\Lipiodol\\Data"

### other CT

In [75]:
for seg in glob.glob(join(src_dir,"tumor 24h_*")):
    patient_id = seg[seg.rfind('_Pat')+4:seg.rfind('.')]
    if len(patient_id) == 1:
        patient_id = "0"+patient_id
    dest = join(target_dir, patient_id)
    #if not exists(join(dest,basename(seg))):
    shutil.move(seg, dest)
    #print(seg, dest)

In [27]:
ct_img = tr.apply_window(ct_img)
ct_img -= np.amin(ct_img)

In [ ]:
for folder in glob.glob(join(img_dir,"*","*MRI 30*")):
    lesion_id = basename(dirname(folder))[:2]
    shutil.copytree(folder, join(target_dir, lesion_id, "MRI-30d"))

In [ ]:
for seg in glob.glob(join(target_dir,"*","*CT*")):
    contents = os.listdir(seg)
    filterstr = ' 1 5 '
    target_folder = [x for x in contents if filterstr in x]
    others = [x for x in contents if filterstr not in x]
    if len(target_folder) == 1:
        for o in others:
            shutil.rmtree(join(seg, o))

# BLMRI

In [12]:
path_dict = {}
patient_id = '04'
for path in glob.glob(join(target_dir, patient_id, "MRI-BL","*","*")):
    header = hf.dcm_load_header(path)
    if len(header) > 0:
        series_descript = header[0][('0008', '103e')].value
        path_dict[path] = series_descript
        series_descript = series_descript.replace("/","-")
        os.rename(path, join(target_dir,patient_id, "MRI-BL", series_descript))

In [ ]:
dcm_paths = [x for x in glob.glob(join(target_dir, "0Ready", "*", "MRI-BL","*")) if x.find('.')==-1]
for dcm_path in dcm_paths:
    if not exists(splitext(dcm_path)[0] + ".nii.gz"):
        try:
            hf.dcm_load(dcm_path, save_path=splitext(dcm_path)[0] + ".nii.gz")
        except:
            print(dcm_path)

In [7]:
art_paths = [x for x in glob.glob(join(target_dir, "0Ready", "*", "MRI-BL","*.nii.gz")) if '20s' in x or 'Ph1-Ax' in x]
for path in art_paths:
    os.rename(path, join(dirname(path), "t1-art.nii.gz"))

# Masks

In [299]:
maskpaths = [x for x in glob.glob(join(target_dir, "0Ready", "*", "*.ids")) if 'liver bl' in x]
#art_paths = glob.glob(join(target_dir, "0Ready", "*", "MRI-BL", "t1-art.nii.gz"))
for path in maskpaths:
    if exists(join(dirname(path), "MRI-BL", "truth.nii.gz")):
        continue
    art_path = join(dirname(path), "MRI-BL", "t1-art.nii.gz")
    img, img_dims = hf.nii_load(art_path, True, True)
    mask, dims = masks.get_mask(path, img_dims=img_dims, img_shape=img.shape)
    hf.save_nii(mask, join(dirname(path), "MRI-BL", "truth.nii.gz"), dims)

[32.192, 93.187, 31.421] [3.183, 3.183, 3.183]


In [283]:
importlib.reload(masks)

<module 'niftiutils.masks' from 'c:\\users\\clinton\\documents\\niftiutils\\niftiutils\\masks.py'>

In [63]:
for patient in os.listdir(img_dir):
    if not exists(join(target_dir, patient[:2], "CT")):
        paths = [x for x in os.listdir(os.path.join(img_dir, patient)) if x.endswith('CT') or x[:-2].endswith('CT')]
        for path in paths:
            subpaths = [x for x in os.listdir(os.path.join(img_dir, patient, path)) if x.find('.')==-1]
            if len(subpaths) > 1:
                print(patient, path)
            elif len(subpaths) == 1:
                if not exists(join(target_dir, patient[:2], "CT")):
                    shutil.copytree(join(img_dir, patient, path, subpaths[0]), join(target_dir, patient[:2], "CT"))
                else:
                    shutil.copytree(join(img_dir, patient, path, subpaths[0]), join(target_dir, patient[:2], "CT2"))
        #print(os.path.join(img_dir, patient, ct24_path))

In [49]:
for patient in os.listdir(img_dir):
    blmr_paths = [x for x in os.listdir(os.path.join(img_dir, patient)) if x.endswith('MRI baseline')]
    #os.makedirs(join(target_dir, patient))
    for blmr_path in blmr_paths:
        shutil.copytree(join(img_dir, patient, blmr_path), join(target_dir, patient, "MRI-BL"))
    #print(os.path.join(img_dir, patient, ct24_path))